In [1]:
import requests
import lxml.html as lh
import pandas as pd
import re 

In [2]:
url = 'https://en.wikipedia.org/wiki/Bernd_Leno'

def getNationalTeamTable(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tables = doc.xpath('//table')
    for table in tables:
        captions = table.xpath('//caption')
        for caption in captions:
            text = caption.text_content()
            if "Appearances and goals by national team and year" in text:
                return table

# TODO: Better Look Up
def getAppsForPlayer(wiki_name):
    url = 'https://en.wikipedia.org/wiki/'+wiki_name
    table = getNationalTeamTable(url)
    if table:
        rows = table.xpath('//tr')
        i=0
        for row in rows:
            i=i+1
            if "Total" in row.text_content():
                if len(row) == 3:
                    total = row[1]
                    return int(total.text_content()), url
    print('Error Getting Apps for: ' + url)
    return -1 ,url

In [3]:
def getPlayerRows(url):
    playerRows = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    rows = doc.xpath('//tr')
    for row in rows:
        if len(row) == 12:
            name_cell = row[3]
            nationality_cell = row[2]
            text = name_cell.text_content().strip()
            skip = text.isnumeric() or len(text) == 0 or text == 'Name'
            if not skip:
                playerRows.append((name_cell,nationality_cell))
    if len(playerRows) == 0:
        print('Error Getting Players from: ' + url)
    else:
        print('Jheeeze found {} players'.format(len(playerRows)))
    return playerRows
    
    
def getWikiLink(cell):
    links = list(cell.iterlinks())
    if len(links)==0:
        return 'ERROR'
    return links[0][2].replace('/wiki/','')


def getWikiDetails(cells):
    player = cells[0]
    country_cell = cells[1]
    display_name = player.text_content()
#     links = list(player.iterlinks())
#     wiki_name = links[0][2].replace('/wiki/','')
    return display_name, getWikiLink(player), getWikiLink(country_cell)

def addPlayerApps(name_tuple):
    display_name = name_tuple[0]
    wiki_name = name_tuple[1]
    country = name_tuple[2]
    (apps,url) = getAppsForPlayer(wiki_name)
    return display_name, wiki_name, country, apps, url

In [4]:
def addPlayerApps(name_tuple):
    display_name = name_tuple[0]
    wiki_name = name_tuple[1]
    country = name_tuple[2]
    (apps,url) = getAppsForPlayer(wiki_name)
    return display_name, wiki_name, country, apps, url

In [5]:
def getAppearanceArray(url):
    playerRows = getPlayerRows(url)
    linksArray = [getWikiDetails(player) for player in playerRows]
    appsArray = [addPlayerApps(names) for names in linksArray]
    return appsArray

In [6]:
def getTeams(url):
    teams = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    rows = doc.xpath('//tr')
    for row in rows:
        if len(row) == 4:
            cell = row[0]
            text = cell.text_content().strip()
            skip = text == 'Team' or text == 'Rank' or text.isnumeric()
            if not skip:
                teams.append(cell)
    return teams


def addTeamToEachItem(player,teamName,squad_url):
    (display_name, wiki_name, country, apps, url) = player
    return teamName,squad_url,display_name, wiki_name, country, apps, url

In [7]:
def fuckedIt(name):
    return 'Fucked It','should be wiki link for'+name,'but Im weird', None, None, None, None

In [ ]:
def toDf(data):
    pd.DataFrame(data, columns = ['teamName','squad_url','display_name', 'wiki_name', 'country', 'apps', 'url']) 

In [8]:
def getDataFrameForLeague(url):
    teams = getTeams(url)
    dfData = []
    for team in teams:
        teamDisplayName = team.text_content()
        wiki_name = getWikiLink(team)
        if wiki_name == 'ERROR':
            dfData = dfData + [fuckedIt(teamDisplayName)]
        else:
            squad_url = 'https://en.wikipedia.org/wiki/2019-20_'+wiki_name+'_season'
            print('Doing Look up for: '+ teamDisplayName)
            appearanceArray = getAppearanceArray(squad_url)
            updatedArray = [addTeamToEachItem(player,teamDisplayName,squad_url) for player in appearanceArray]
            dfData = dfData + updatedArray
    return toDf(dfData)

In [9]:
# url = 'https://en.wikipedia.org/wiki/2020%E2%80%9321_Premier_League#Teams'
# data = getDataFrameForLeague(url)
# print(len(data))

In [10]:
def writeDataToFile(df,fileName):
    df.to_csv('../data/'+fileName,index=False)
    print(df.head())

In [11]:
# writeDataToFile(data,"PremSample.csv")

In [12]:
import csv

with open('../data/leagues.csv', newline='') as csvfile:
    leagues = list(csv.reader(csvfile))

print(leagues)

[['SPAIN', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_La_Liga'], ['ENGLAND', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Premier_League'], ['GERMANY', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Bundesliga'], ['ITALY', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Serie_A'], ['FRANCE', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Ligue_1'], ['PORTUGAL', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Primeira_Liga'], ['RUSSIA', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Russian_Premier_League'], ['BELGIUM', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Belgian_First_Division_A'], ['UKRAINE', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Ukrainian_Premier_League'], ['NETHERLANDS', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Eredivisie'], ['TURKEY', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_S%C3%BCper_Lig'], ['AUSTRIA', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Austrian_Football_Bundesliga'], ['DENMARK', 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Danish

In [13]:

def getLeagueObject(league):
    country = league[0]
    url = league[1]
    leagueData = getDataFrameForLeague(url)
    thisDict = {
        "country":country,
        "url":url,
        "data":leagueData
    }
    return thisDict

    

In [14]:
dictionaries = [getLeagueObject(league) for league in leagues]


Doing Look up for: Alavés
Jheeeze found 1 players
Error Getting Apps for: https://en.wikipedia.org/wiki/2019%E2%80%9320_Copa_del_Rey#First_round
Doing Look up for: Athletic Bilbao
Jheeeze found 1 players
Error Getting Apps for: https://en.wikipedia.org/wiki/ERROR
Doing Look up for: Atlético Madrid
Jheeeze found 44 players
Error Getting Apps for: https://en.wikipedia.org/wiki/Antonio_Ad%C3%A1n
Error Getting Apps for: https://en.wikipedia.org/wiki/Jos%C3%A9_Mar%C3%ADa_Gim%C3%A9nez
Error Getting Apps for: https://en.wikipedia.org/wiki/Santiago_Arias
Error Getting Apps for: https://en.wikipedia.org/wiki/Thomas_Partey
Error Getting Apps for: https://en.wikipedia.org/wiki/Koke_(footballer,_born_1992)


<ipython-input-2-60738d804e64>:18: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if table:


Error Getting Apps for: https://en.wikipedia.org/wiki/Sa%C3%BAl_%C3%91%C3%ADguez
Error Getting Apps for: https://en.wikipedia.org/wiki/Jan_Oblak
Error Getting Apps for: https://en.wikipedia.org/wiki/Marcos_Llorente
Error Getting Apps for: https://en.wikipedia.org/wiki/Stefan_Savi%C4%87
Error Getting Apps for: https://en.wikipedia.org/wiki/H%C3%A9ctor_Herrera
Error Getting Apps for: https://en.wikipedia.org/wiki/Ivan_%C5%A0aponji%C4%87
Error Getting Apps for: https://en.wikipedia.org/wiki/Felipe_(footballer,_born_1989)
Error Getting Apps for: https://en.wikipedia.org/wiki/Diego_Costa
Error Getting Apps for: https://en.wikipedia.org/wiki/Vitolo_(footballer,_born_1989)
Error Getting Apps for: https://en.wikipedia.org/wiki/Yannick_Carrasco
Error Getting Apps for: https://en.wikipedia.org/wiki/Mario_Hermoso
Error Getting Apps for: https://en.wikipedia.org/wiki/%C5%A0ime_Vrsaljko
Error Getting Apps for: https://en.wikipedia.org/wiki/Andr%C3%A9_Moreira
Error Getting Apps for: https://en.wikip

Error Getting Apps for: https://en.wikipedia.org/wiki/Fay%C3%A7al_Fajr
Error Getting Apps for: https://en.wikipedia.org/wiki/Jason_(Spanish_footballer)
Error Getting Apps for: https://en.wikipedia.org/wiki/David_Timor
Error Getting Apps for: https://en.wikipedia.org/wiki/Jaime_Mata
Error Getting Apps for: https://en.wikipedia.org/wiki/%C3%81ngel_Rodr%C3%ADguez_(Spanish_footballer)
Error Getting Apps for: https://en.wikipedia.org/wiki/Amath_Ndiaye
Error Getting Apps for: https://en.wikipedia.org/wiki/Jorge_Molina_Vidal
Error Getting Apps for: https://en.wikipedia.org/wiki/Hugo_Duro
Error Getting Apps for: https://en.wikipedia.org/wiki/Vitorino_Antunes
Error Getting Apps for: https://en.wikipedia.org/wiki/Bruno_Gonz%C3%A1lez_Cabrera
Error Getting Apps for: https://en.wikipedia.org/wiki/Markel_Bergara
Error Getting Apps for: https://en.wikipedia.org/wiki/Leandro_Cabrera
Error Getting Apps for: https://en.wikipedia.org/wiki/Enric_Gallego
Error Getting Apps for: https://en.wikipedia.org/wik

Jheeeze found 32 players
Error Getting Apps for: https://en.wikipedia.org/wiki/Kieran_Tierney
Error Getting Apps for: https://en.wikipedia.org/wiki/Lucas_Torreira
Error Getting Apps for: https://en.wikipedia.org/wiki/Rob_Holding
Error Getting Apps for: https://en.wikipedia.org/wiki/Pablo_Mar%C3%AD
Error Getting Apps for: https://en.wikipedia.org/wiki/Reiss_Nelson
Error Getting Apps for: https://en.wikipedia.org/wiki/Emiliano_Mart%C3%ADnez
Error Getting Apps for: https://en.wikipedia.org/wiki/Konstantinos_Mavropanos
Error Getting Apps for: https://en.wikipedia.org/wiki/Joe_Willock
Error Getting Apps for: https://en.wikipedia.org/wiki/Matteo_Guendouzi
Error Getting Apps for: https://en.wikipedia.org/wiki/Eddie_Nketiah
Error Getting Apps for: https://en.wikipedia.org/wiki/Emile_Smith_Rowe
Error Getting Apps for: https://en.wikipedia.org/wiki/Matt_Macey
Error Getting Apps for: https://en.wikipedia.org/wiki/Gabriel_Martinelli
Error Getting Apps for: https://en.wikipedia.org/wiki/William_Sal

...

Error Getting Apps for: https://en.wikipedia.org/wiki/Andrija_Radulovi%C4%87_(footballer)
Error Getting Apps for: https://en.wikipedia.org/wiki/Jos%C3%A9_Ca%C3%B1as
Error Getting Apps for: https://en.wikipedia.org/wiki/Milan_Pavkov
Error Getting Apps for: https://en.wikipedia.org/wiki/Richmond_Boakye
Error Getting Apps for: https://en.wikipedia.org/wiki/Toman%C3%A9
Error Getting Apps for: https://en.wikipedia.org/wiki/El_Fardou_Ben_Nabouhane
Error Getting Apps for: https://en.wikipedia.org/wiki/Borisav_Burmaz
Error Getting Apps for: https://en.wikipedia.org/wiki/Aleksa_Vukanovi%C4%87
Error Getting Apps for: https://en.wikipedia.org/wiki/Marko_Marin
Error Getting Apps for: https://en.wikipedia.org/wiki/Jander_Ribeiro_Santana
Error Getting Apps for: https://en.wikipedia.org/wiki/Filip_Stojkovi%C4%87
Error Getting Apps for: https://en.wikipedia.org/wiki/Milo%C5%A1_Pantovi%C4%87_(footballer,_born_2002)
Error Getting Apps for: https://en.wikipedia.org/wiki/Milan_Jevtovi%C4%87
Error Getting 

Error Getting Apps for: https://en.wikipedia.org/wiki/Joe_Kuzminsky
Doing Look up for: Maccabi Netanya

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Maccabi_Netanya_F.C._season
Doing Look up for: Maccabi Tel Aviv

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Maccabi_Tel_Aviv_F.C._season
Doing Look up for: Sektzia Nes Tziona

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Sektzia_Nes_Tziona_F.C._season
Doing Look up for: Budapest Honvéd

Jheeeze found 69 players
Error Getting Apps for: https://en.wikipedia.org/wiki/Robi_Levkovich
Error Getting Apps for: https://en.wikipedia.org/wiki/Mohamed_Mezghrani
Error Getting Apps for: https://en.wikipedia.org/wiki/Eke_Uzoma
Error Getting Apps for: https://en.wikipedia.org/wiki/D%C3%A1vid_K%C3%A1lnoki-Kis
Error Getting Apps for: https://en.wikipedia.org/wiki/D%C3%A1niel_Gazdag
Error Getting Apps for: https://en.wikipedia.org/wiki/Bence_Batik
Error Getting Apps for: https://en.wikipedia.or

Jheeeze found 1 players
Error Getting Apps for: https://en.wikipedia.org/wiki/ERROR
Doing Look up for: River Plate

Jheeeze found 1 players
Error Getting Apps for: https://en.wikipedia.org/wiki/ERROR
Doing Look up for: Rosario Central

Jheeeze found 1 players
Error Getting Apps for: https://en.wikipedia.org/wiki/ERROR
Doing Look up for: San Lorenzo

Jheeeze found 1 players
Error Getting Apps for: https://en.wikipedia.org/wiki/ERROR
Doing Look up for: Talleres (C)

Jheeeze found 1 players
Error Getting Apps for: https://en.wikipedia.org/wiki/ERROR
Doing Look up for: Unión

Jheeeze found 1 players
Error Getting Apps for: https://en.wikipedia.org/wiki/ERROR
Doing Look up for: Vélez Sarsfield

Jheeeze found 1 players
Error Getting Apps for: https://en.wikipedia.org/wiki/ERROR
Doing Look up for: Audax Italiano
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Audax_Italiano_season
Doing Look up for: Cobresal
Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_C

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Al-Adalah_F.C._season
Doing Look up for: Al-Ahli

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Al-Ahli_Saudi_FC_season
Doing Look up for: Al-Ettifaq

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Ettifaq_FC_season
Doing Look up for: Al-Faisaly

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Al-Faisaly_FC_season
Doing Look up for: Al-Fateh

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Al-Fateh_SC_season
Doing Look up for: Al-Fayha

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Al-Fayha_FC_season
Doing Look up for: Al-Hazem

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Al-Hazem_F.C._season
Doing Look up for: Al-Hilal

Jheeeze found 1 players
Error Getting Apps for: https://en.wikipedia.org/wiki/ERROR
Doing Look up for: Al-Ittihad

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Ittihad

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Inter-R%C3%A9gions_Division_season
Doing Look up for: Régional I

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Ligue_R%C3%A9gional_I_season
Doing Look up for: Régional I

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Ligue_R%C3%A9gional_I_season
Doing Look up for: Régional I

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Ligue_R%C3%A9gional_I_season
Doing Look up for: Régional I

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Ligue_R%C3%A9gional_I_season
Doing Look up for: Régional I

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Ligue_R%C3%A9gional_I_season
Doing Look up for: Régional I

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Ligue_R%C3%A9gional_I_season
Doing Look up for: Régional I

Error Getting Players from: https://en.wikipedia.org/wiki/2019-20_Ligue_R%C3%A9gional_I_season
Doing Look up for

In [20]:
for dictionary in dictionaries:
    print("{} : {}".format(len(dictionary["data"]),dictionary["url"],))

272 : https://en.wikipedia.org/wiki/2019%E2%80%9320_La_Liga
326 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Premier_League
6 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Bundesliga
9 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Serie_A
0 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Ligue_1
2 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Primeira_Liga
58 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Russian_Premier_League
17 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Belgian_First_Division_A
2 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Ukrainian_Premier_League
74 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Eredivisie
87 : https://en.wikipedia.org/wiki/2019%E2%80%9320_S%C3%BCper_Lig
32 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Austrian_Football_Bundesliga
1 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Danish_Superliga
5 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Scottish_Premiership
3 : https://en.wikipedia.org/wiki/2019%E2%80%9320_Czech_First